In [42]:
import pandas as pd

# Load pickle data into a DataFrame
df = pd.read_pickle('../_3_data_preprocessing/output/serialized_review_dataframe.pkl')

# Display the DataFrame
print(df)  #Print the first few rows to verify


                                            review_text  F  BR  AU  FI  IR  A  \
0     {'input_ids': [[tensor(101), tensor(2191), ten...  0   0   0   0   0  0   
1     {'input_ids': [[tensor(101), tensor(3898), ten...  0   0   0   0   0  0   
2     {'input_ids': [[tensor(101), tensor(6203), ten...  0   0   0   0   1  0   
3     {'input_ids': [[tensor(101), tensor(10439), te...  0   1   0   0   0  0   
4     {'input_ids': [[tensor(101), tensor(2052), ten...  0   1   0   0   0  0   
...                                                 ... ..  ..  ..  ..  .. ..   
1229  {'input_ids': [[tensor(101), tensor(2204), ten...  1   0   0   0   0  0   
1230  {'input_ids': [[tensor(101), tensor(8670), ten...  0   1   0   0   0  0   
1231  {'input_ids': [[tensor(101), tensor(6904), ten...  0   0   0   0   1  0   
1232  {'input_ids': [[tensor(101), tensor(3291), ten...  0   0   0   0   0  0   
1233  {'input_ids': [[tensor(101), tensor(6592), ten...  0   0   0   0   0  0   

      L  LF  MN  O  PE  SC 

In [37]:
from sklearn.model_selection import train_test_split

# Assume your DataFrame is named df
# Extract features (review_text) and labels (F, BR, AU, FI, IR, A, L, LF, MN, O, PE, SC, SE, US, PO)
features = df['review_text']
labels = df[['F', 'BR', 'AU', 'FI', 'IR', 'A', 'L', 'LF', 'MN', 'O', 'PE', 'SC', 'SE', 'US', 'PO']]

# Split the dataset into training and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Now you have your training and validation sets ready
# train_features and train_labels are for training
# val_features and val_labels are for validation


In [56]:
list(train_features)[0]

{'input_ids': tensor([[  101,  2307, 10439,  4299,  2134,  1043, 15909,  2818,  6686,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
train_labels

,F,BR,AU,FI,IR,A,L,LF,MN,O,PE,SC,SE,US,PO
869,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
728,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
803,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
644,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
813,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1095,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
860,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [28]:
list(train_features)[0][1][0].shape[1]

10

In [58]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Example data
train_features, train_labels = train_features[:100], train_labels[:100]  # Example subset of data

# Tokenize input data
train_input = {'input_ids':[obj["input_ids"] for obj in list(train_features)],'attention_mask':[obj["attention_mask"] for obj in list(train_features)]}

# Define model architecture
input_ids = Input(shape=(train_input["input_ids"].shape[1],), dtype=tf.int32)
attention_mask = Input(shape=(train_input["attention_mask"].shape[1],), dtype=tf.int32)

bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]
output = Dense(15, activation='sigmoid')(bert_output)

model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x={'input_ids': train_input["input_ids"], 'attention_mask': train_input["attention_mask"]},
          y=train_labels,
          epochs=5,
          batch_size=32,
          validation_split=0.2)

# Evaluate the model
# Evaluate the model
val_input = {'input_ids':[obj["input_ids"] for obj in list(val_features)],'attention_mask':[obj["attention_mask"] for obj in list(val_features)]}
results = model.evaluate(x={'input_ids': val_input["input_ids"], 'attention_mask': val_input["attention_mask"]},
                         y=val_labels)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

AttributeError: 'list' object has no attribute 'shape'